# Agricultural Application: Weather Prediction and Risk Assessment

## Real-World Problem

Farmers must make critical decisions under weather uncertainty:
- **When to plant**: Risk of late frost vs. optimal growth window
- **When to harvest**: Rain damage vs. crop maturity
- **Whether to irrigate**: Cost vs. drought probability

This notebook uses conditional probability and historical data for optimal timing decisions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import sys
sys.path.append('../2_from_scratch')
import probability_functions as pf

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Ready for weather analysis!")

## Problem 1: Spring Planting Decision

**Question**: Plant early (higher yields if no frost) or wait (safer but lower yields)?

In [ ]:
# Historical data: last frost dates
last_frost_dates = np.array([
    95, 102, 98, 105, 92, 108, 94, 100, 97, 103,  # Days from Jan 1
    99, 96, 101, 104, 93, 107, 95, 98, 102, 100
])

# Planting options
early_plant_date = 98  # April 8 (day 98)
safe_plant_date = 110  # April 20 (day 110)

# Calculate frost probability
P_frost_after_early = (last_frost_dates > early_plant_date).mean()
P_frost_after_safe = (last_frost_dates > safe_plant_date).mean()

print("Spring Planting Decision Analysis")
print("="*70)
print(f"\nHistorical data: {len(last_frost_dates)} years")
print(f"Average last frost: Day {last_frost_dates.mean():.0f} (April {int(last_frost_dates.mean())-90})")
print(f"Std deviation: {last_frost_dates.std():.1f} days")

print(f"\nPlanting Options:")
print(f"  Early (Day {early_plant_date}): P(frost damage) = {P_frost_after_early:.0%}")
print(f"  Safe (Day {safe_plant_date}): P(frost damage) = {P_frost_after_safe:.0%}")

# Economic analysis
yield_early_no_frost = 110  # bushels/acre
yield_early_with_frost = 40  # damaged
yield_safe = 95  # lower yield but no frost risk

expected_yield_early = (
    (1 - P_frost_after_early) * yield_early_no_frost + 
    P_frost_after_early * yield_early_with_frost
)

print(f"\nExpected Yields:")
print(f"  Early planting: {expected_yield_early:.1f} bu/ac")
print(f"  Safe planting: {yield_safe:.1f} bu/ac")

if expected_yield_early > yield_safe:
    advantage = expected_yield_early - yield_safe
    print(f"\n✓ RECOMMENDATION: Plant early (+{advantage:.1f} bu/ac expected)")
else:
    print(f"\n✓ RECOMMENDATION: Wait for safe date (higher expected return)")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.hist(last_frost_dates, bins=15, edgecolor='black', alpha=0.7)
ax1.axvline(early_plant_date, color='orange', linestyle='--', linewidth=2, label=f'Early ({early_plant_date})')
ax1.axvline(safe_plant_date, color='green', linestyle='--', linewidth=2, label=f'Safe ({safe_plant_date})')
ax1.set_xlabel('Day of Year')
ax1.set_ylabel('Frequency')
ax1.set_title('Historical Last Frost Dates')
ax1.legend()

scenarios = ['Early\nNo Frost', 'Early\nWith Frost', 'Safe\nPlanting']
yields = [yield_early_no_frost, yield_early_with_frost, yield_safe]
probabilities = [1-P_frost_after_early, P_frost_after_early, 1.0]
colors = ['darkgreen', 'red', 'lightgreen']

bars = ax2.bar(scenarios, yields, color=colors, edgecolor='black', linewidth=2, alpha=0.7)
ax2.axhline(expected_yield_early, color='blue', linestyle='--', linewidth=2, 
           label=f'Early Expected: {expected_yield_early:.1f}')
ax2.set_ylabel('Yield (bu/ac)')
ax2.set_title('Yield Scenarios')
ax2.legend()

for i, (y, p) in enumerate(zip(yields, probabilities)):
    if i < 2:
        ax2.text(i, y + 3, f'{y} bu\n(P={p:.0%})', ha='center', fontsize=9, fontweight='bold')
    else:
        ax2.text(i, y + 3, f'{y} bu', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## Problem 2: Conditional Weather Forecasting

Using current conditions to predict future weather

In [ ]:
# Simulated weather transition data
# P(tomorrow | today)
transition_matrix = pd.DataFrame({
    'Sunny_tomorrow': [0.70, 0.40, 0.20],
    'Cloudy_tomorrow': [0.25, 0.40, 0.30],
    'Rainy_tomorrow': [0.05, 0.20, 0.50]
}, index=['Sunny_today', 'Cloudy_today', 'Rainy_today'])

print("Weather Transition Probabilities")
print("="*70)
print("\nP(Tomorrow's Weather | Today's Weather):")
print(transition_matrix)

# Scenario: Currently cloudy, need 3 dry days for harvest
print("\n\nHarvest Planning: Need 3 consecutive dry days")
print("-"*70)
print("Current condition: Cloudy")

# P(3 dry days starting tomorrow | cloudy today)
p_dry_day1 = transition_matrix.loc['Cloudy_today', 'Sunny_tomorrow'] + \
             transition_matrix.loc['Cloudy_today', 'Cloudy_tomorrow']
             
# Assume tomorrow is sunny
p_dry_day2_given_sunny = transition_matrix.loc['Sunny_today', 'Sunny_tomorrow'] + \
                         transition_matrix.loc['Sunny_today', 'Cloudy_tomorrow']

# Approximate P(3 dry days)
p_3_dry_approx = p_dry_day1 * p_dry_day2_given_sunny * p_dry_day2_given_sunny

print(f"\nP(Day 1 dry | cloudy today) = {p_dry_day1:.0%}")
print(f"P(Day 2 dry | sunny) ≈ {p_dry_day2_given_sunny:.0%}")
print(f"P(3 consecutive dry days) ≈ {p_3_dry_approx:.0%}")

if p_3_dry_approx > 0.60:
    print(f"\n✓ Good conditions for harvest planning")
else:
    print(f"\n⚠️  Risky - consider waiting or alternative plans")

## Problem 3: Monte Carlo Weather Simulation

In [ ]:
# Monte Carlo: Simulate growing season weather
n_simulations = 5000
season_length = 120  # days

# Weather parameters
p_rain = 0.25  # Daily rain probability
min_rain_for_drought = 15  # Need at least 15 rainy days

# Simulate
rainy_days = np.random.binomial(season_length, p_rain, n_simulations)
drought_seasons = (rainy_days < min_rain_for_drought).sum()
P_drought = drought_seasons / n_simulations

print("Monte Carlo: Seasonal Drought Risk")
print("="*70)
print(f"Simulations: {n_simulations:,}")
print(f"Season length: {season_length} days")
print(f"Daily rain probability: {p_rain:.0%}")
print(f"\nResults:")
print(f"  Average rainy days: {rainy_days.mean():.1f}")
print(f"  Std deviation: {rainy_days.std():.1f}")
print(f"  P(drought) = P(<{min_rain_for_drought} rainy days) = {P_drought:.1%}")

# Irrigation decision
irrigation_cost = 30000
drought_loss = 150000
expected_loss = P_drought * drought_loss

print(f"\nIrrigation Decision:")
print(f"  Cost: ${irrigation_cost:,}")
print(f"  Expected loss without: ${expected_loss:,.0f}")

if expected_loss > irrigation_cost:
    print(f"  ✓ INVEST in irrigation (saves ${expected_loss - irrigation_cost:,.0f})")
else:
    print(f"  ✗ Don't invest (not cost-effective)")

# Visualize
plt.figure(figsize=(12, 5))
plt.hist(rainy_days, bins=40, edgecolor='black', alpha=0.7, density=True)
plt.axvline(rainy_days.mean(), color='blue', linestyle='--', linewidth=2, 
           label=f'Mean = {rainy_days.mean():.1f}')
plt.axvline(min_rain_for_drought, color='red', linestyle='--', linewidth=2,
           label=f'Drought threshold = {min_rain_for_drought}')
plt.xlabel('Number of Rainy Days')
plt.ylabel('Probability Density')
plt.title(f'Monte Carlo Simulation: Rainy Days in Growing Season (n={n_simulations:,})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n💡 {drought_seasons:,} out of {n_simulations:,} simulated seasons had drought")

## Summary: Weather-Based Decision Making

### Key Lessons

**1. Historical Data Analysis**
- Use past weather patterns to estimate probabilities
- Calculate expected values for competing strategies
- Account for uncertainty in timing decisions

**2. Conditional Probabilities**
- Current weather predicts future weather (Markov property)
- Transition matrices capture weather dependencies
- Sequential probability calculations for multi-day events

**3. Monte Carlo Simulation**
- Model complex seasonal patterns
- Estimate rare event probabilities (drought, flooding)
- Support investment decisions (irrigation, insurance)

**4. Risk Management**
- Compare costs vs. expected losses
- Quantify uncertainty in agricultural planning
- Make data-driven decisions under uncertainty

### Applications
- Planting and harvest timing
- Irrigation scheduling
- Crop insurance decisions
- Resource allocation

### Next
Continue to: `integrated_farming_decisions.ipynb` for comprehensive multi-factor analysis!